# L6a: Incorporating Gene Expression Logic into Flux Balance Analysis
In this lecture, we'll continue our discussion of Flux Balance Analysis (FBA), particularly what the constraints are saying in the flux estimation problem. Last time, we simplified the matreial balance and flux bounds constraints. Today, we'll discuss incorporating gene expression logic into the FBA problem. The key ideas of this lecture are:
* __Flux balance analysis (FBA)__ is a mathematical approach used to analyze the flow of metabolites through a metabolic network. It assumes a steady state where metabolite production, consumption, and transport rates are balanced. The FBA problem is formulated as a linear programming (LP) problem to maximize or minimize fluxes through the network, subject to constraints. 
* __Flux bounds constraints__ limit the range of possible fluxes through a metabolic network. These bounds can incorporate additional information, such as experimental data or prior knowledge about the system, into the FBA problem.
* __Gene expression logic__ can be incorporated into the FBA problem by using gene-protein-reaction (GPR) rules to link gene expression levels to enzyme activity and metabolic fluxes. GPR rules define the relationship between genes, proteins, and reactions in a metabolic network, allowing for the integration of gene expression data into the FBA model.

Lecture notes can be downloaded: [here!](https://github.com/varnerlab/CHEME-5450-Lectures-Spring-2025/blob/main/lectures/week-6/L6a/docs/Notes.pdf)

## A model for flux bounds
The flux bounds are important constraints in flux balance analysis calculations and the convex decomposition of the stoichiometric array. Beyond their role in the flux estimation problem, the flux bounds are _integrative_, i.e., these constraints integrate many types of genetic and biochemical information into the problem. A general model for these bounds is given by:
$$
\begin{align*}
-\delta_{j}\underbrace{\left[{V_{max,j}^{\circ}}\left(\frac{e}{e^{\circ}}\right)\theta_{j}\left(\dots\right){f_{j}\left(\dots\right)}\right]}_{\text{reverse: other functions or parameters?}}\leq\hat{v}_{j}\leq{V_{max,j}^{\circ}}\left(\frac{e}{e^{\circ}}\right)\theta_{j}\left(\dots\right){f_{j}\left(\dots\right)}
\end{align*}
$$
where $V_{max,j}^{\circ}$ denotes the maximum reaction velocity (units: `flux`) computed at some _characteristic enzyme abundance_. Thus, the maximum reaction velocity is given by:
$$
V_{max,j}^{\circ} \equiv k_{cat,j}^{\circ}e^{\circ}
$$
where $k_{cat,j}$ is the catalytic constant or turnover number for the enzyme (units: `1/time`) and $e^{\circ}$ is a characteristic enzyme abundance (units: `concentration`). The term $\left(e/e^{\circ}\right)$ is a correction to account for the _actual_ enzyme abundance catalyzing the reaction (units: `dimensionless`). The $\theta_{j}\left(\dots\right)\in\left[0,1\right]$ is the current fraction of maximial enzyme activity of enzyme $e$ in reaction $j$. The activity model $\theta_{j}\left(\dots\right)$ describes [allosteric effects](https://en.wikipedia.org/wiki/Allosteric_regulation) on the reaction rate, and is a function of the regulatory and the chemical state of the system, the concentration of substrates, products, and cofactors (units: `dimensionless`).
Finally, the $f_{j}\left(\dots\right)$ is a function describing the substrate (reactants) dependence of the reaction rate $j$ (units: `dimensionless`). 

* __Parameters__: We need estimates for the $k_{cat,j}^{\circ}$ for all enzymes in the system we are interested in and a _reasonable policy_ for specifying a characteristic value for $e^{\circ}$. In addition, the $\theta_{j}\left(\dots\right)$ and $f_{j}\left(\dots\right)$ models can also have associated parameters, e.g., saturation or binding constants, etc. Thus, we need to estimate these from literature studies or experimental data.
* __Reversibility__: Next, we need to estimate the binary direction parameter $\delta_{j}\in\left\{0,1\right\}$. The value of $\delta_{j}$ describes the reversibility of reaction $j$; if reaction $j$ is __reversible__ $\delta_{j}=1$. If reaction $j$ is __irreversible__ $\delta_{j}=0$

### Simplfied bounds model
Let's initially assume that $(e/e^{\circ})\sim{1}$, there are no allosteric inputs $\theta_{j}\left(\dots\right)\sim{1}$, and the substrates are saturating $f_{j}\left(\dots\right)\sim{1}$. 
Then, the flux bounds are given by:
$$
\begin{align*}
-\delta_{j}V_{max,j}^{\circ}\leq{\hat{v}_{j}}\leq{V_{max,j}^{\circ}}
\end{align*}
$$
This is a simple model for the flux bounds. It is easy to see that the flux bounds are a function of the maximum reaction velocity, the catalytic constant or turnover number, and our assumed value of a characteristic enzyme abundance.

#### Turnover numbers
The turnover number, $k_{cat}$, measures an enzyme's catalytic activity, defined as the number of substrate molecules converted to product per enzyme molecule per unit time. Units are typically `1/time` (e.g., `1/s` or `1/min`). Values can be obtained from primary literature or databases like [BRENDA](https://www.brenda-enzymes.org/):

* [Antje Chang et al., BRENDA, the ELIXIR core data resource in 2021: new developments and updates, Nucleic Acids Research, Volume 49, Issue D1, 8 January 2021, Pages D498–D508, https://doi.org/10.1093/nar/gkaa1025](https://academic.oup.com/nar/article/49/D1/D498/5992283)

__Example__: Use [BRENDA](https://www.brenda-enzymes.org/) to find turnover number for: Arginase (EC 3.5.3.1) in humans.

#### Reversibility
The second thing we need to estimate is the reversibility parameter.
The reversibility parameter $\delta_{j}$ can be computed in several ways. For example, [one method in the literature](https://pubmed.ncbi.nlm.nih.gov/27159581/) is to use the sign of Gibbs reaction energy:
$$
\begin{equation*}
\delta_{i} = \begin{cases}
0 & \text{if }\text{sign}\left(\Delta{G}^{\circ} - \Delta{G}^{\star}\right)= -1 \quad\text{irreversible} \\
1 & \text{if }\text{sign}\left(\Delta{G}^{\circ} - \Delta{G}^{\star}\right)= +1 \quad\text{reversible}
\end{cases}
\end{equation*}
$$
where $\Delta{G}^{\circ}$ is the [standard Gibbs free energy change of the reaction](https://en.wikipedia.org/wiki/Gibbs_free_energy#Gibbs_free_energy_of_reactions), and $\Delta{G}^{\star}$ is a threshold value (hyperparameter). The threshold value can be set to zero or some other value. Alternatively, the value of $\delta_{j}$ can be assigned based upon a cutoff $K^{\star}$ on the equilibrium constant:
$$
\begin{equation*}
\delta_{i} = \begin{cases}
0 & \text{if }K_{eq}>\,K^{\star}\quad\text{irreversible} \\
1 & \text{if }K_{eq}\leq\,K^{\star}\quad\text{reversible}
\end{cases}
\end{equation*}
$$
where you specify the value $K^{\star}$ based upon some intution or other criteria. We can compute the $\Delta{G}^{\circ}$ values using [eQuilibrator](https://equilibrator.weizmann.ac.il):
* [Beber ME, Gollub MG, Mozaffari D, Shebek KM, Flamholz AI, Milo R, Noor E. eQuilibrator 3.0: a database solution for thermodynamic constant estimation. Nucleic Acids Res. 2022 Jan 7;50(D1): D603-D609. doi: 10.1093/nar/gkab1106. PMID: 34850162; PMCID: PMC8728285.](https://pubmed.ncbi.nlm.nih.gov/34850162/)

The [eQuilibrator application programming interface](https://equilibrator.weizmann.ac.il) is a tool for thermodynamic calculations in biological reaction networks. It was developed by the [Milo lab](https://www.weizmann.ac.il/plants/Milo/) at the Weizmann Institute in Rehovot, Israel. The [`eQuilibrator.jl` package](https://github.com/stelmo/eQuilibrator.jl) is a [Julia](https://julialang.org) wrapper around eQuilibrator (which is written in Python). 

__Example__: Use [eQuilibrator](https://equilibrator.weizmann.ac.il) to find the $\delta_{j}$ values for: Arginase (EC 3.5.3.1) in humans, where we assume a threshold value of $\Delta{G}^{\star}=-5.0$ kJ/mol (__hmmm__: where did this come from)?

## PS2 (Preview): Flux Balance Analysis of the Urea Cycle in HL-60 Cells
In problem set 2 (PS2), we will explore the urea cycle in HL-60 cells using flux balance analysis. The [urea cycle](https://www.kegg.jp/pathway/hsa00220) is a crucial metabolic pathway that converts toxic ammonia into urea for excretion. While the urea cycle's role in [HL-60 cells, a human promyelocytic leukemia cell line](https://www.atcc.org/products/ccl-240?matchtype=b&network=g&device=c&adposition=&keyword=hl60%20cell%20line%20atcc&gad_source=1&gbraid=0AAAAADR6fpoOXsp8U8fXLd_E6sLTcwv24&gclid=CjwKCAiA5eC9BhAuEiwA3CKwQm0C1oE5_JjTpJ24VnTjZUZQVLivpPxmufDo7HdH5v3hN1XKnEf3ExoCvhwQAvD_BwE), is not directly established, these cells exhibit alterations in protein levels and proliferation rates when exposed to various compounds, which may indirectly affect nitrogen metabolism and related pathways.

* __Tasks__: We'll construct [a simplified model of the urea cycle](https://github.com/varnerlab/CHEME-5450-Lectures-Spring-2025/blob/main/lectures/week-5/L5c/docs/figs/Fig-Urea-cycle-Schematic.pdf), analyze its structure, determining reversibility of the reactions, some estimates for the bounds, and then compute the flux distribution through the network under different assumptions.

### References
1. [Al-Otaibi NAS, Cassoli JS, Martins-de-Souza D, Slater NKH, Rahmoune H. Human leukemia cells (HL-60) proteomic and biological signatures underpinning cryo-damage are differentially modulated by novel cryo-additives. Gigascience. 2019 Mar 1;8(3):giy155. doi: 10.1093/gigascience/giy155. PMID: 30535373; PMCID: PMC6394207.](https://pmc.ncbi.nlm.nih.gov/articles/PMC6394207/)
2. [Figarola JL, Weng Y, Lincoln C, Horne D, Rahbar S. Novel dichlorophenyl urea compounds inhibit proliferation of human leukemia HL-60 cells by inducing cell cycle arrest, differentiation and apoptosis. Invest New Drugs. 2012 Aug;30(4):1413-25. doi: 10.1007/s10637-011-9711-8. Epub 2011 Jul 5. PMID: 21728022.](https://pubmed.ncbi.nlm.nih.gov/21728022/)
3. [Caldwell RW, Rodriguez PC, Toque HA, Narayanan SP, Caldwell RB. Arginase: A Multifaceted Enzyme Important in Health and Disease. Physiol Rev. 2018 Apr 1;98(2):641-665. doi: 10.1152/physrev.00037.2016. PMID: 29412048; PMCID: PMC5966718.](https://pmc.ncbi.nlm.nih.gov/articles/PMC5966718/)

### Setup, Data, and Prerequisites
We set up the computational environment by including the `Include.jl` file, loading any needed resources, such as sample datasets, and setting up any required constants. 
* The `Include.jl` file also loads external packages, various functions that we will use in the exercise, and custom types to model the components of our problem. It checks for a `Manifest.toml` file; if it finds one, packages are loaded. Other packages are downloaded and then loaded.

In [7]:
include("Include.jl");

__Build the model__. To store all the problem data, we created [the `MyPrimalFluxBalanceAnalysisCalculationModel` type](src/Types.jl). Let's build one of these objects for our problem and store it in the `model::MyPrimalFluxBalanceAnalysisCalculationModel` variable. We also return the `rd::Dict{String, String}` dictionary, which maps the reaction name field (key) to the reaction string (value).
* __Builder (or factory) pattern__: For all custom types that we make, we'll use something like [the builder software pattern](https://en.wikipedia.org/wiki/Builder_pattern) to construct and initialize these objects. The calling syntax will be the same for all types: [a `build(...)` method](src/Factory.jl) will take the kind of thing we want to build in the first argument, and the data needed to make that type as [a `NamedTuple` instance](https://docs.julialang.org/en/v1/base/base/#Core.NamedTuple) in the second argument.
* __What's the story with the `let` block__? A [let block](https://docs.julialang.org/en/v1/manual/variables-and-scoping/#Let-Blocks) creates a new hard scope and new variable bindings each time they run. Thus, they act like a private scratch space, where data comes in (is captured by the block), but only what we want to be exposed comes out. 

In [9]:
model, rd = let

    # first, load the reaction file - and process it
    listofreactions = read_reaction_file(joinpath(_PATH_TO_DATA, "Network.net")); # load the reactions from the VFF reaction file
    S, species, reactions, rd = build_stoichiometric_matrix(listofreactions); # Builds the stochiometric matrix, species list, and the reactions list
    boundsarray = build_default_bounds_array(listofreactions); # Builds a default bounds model using the flat file flags

    # build the FBA model -
    model = build(MyPrimalFluxBalanceAnalysisCalculationModel, (
        S = S, # stoichiometric matrix
        fluxbounds = boundsarray, # these are the *default* bounds, we'll need to update with new info if we have it
        species = species, # list of species. The rows of S are in this order
        reactions = reactions, # list of reactions. The cols of S are in this order
        objective = length(reactions) |> R -> zeros(R), # this is empty, we'll need to set this
    ));

    # return -
    model, rd
end;

`Unhide` the code block below to see how we build a table of the reactions in the model [using the `pretty_tables(...)` method exported from the `PrettyTables.jl` package](https://github.com/ronisbr/PrettyTables.jl).

In [11]:
let
    df = DataFrame()
    reactions = model.reactions;

    for i ∈ eachindex(reactions)
        reactionstring = reactions[i] |> key -> rd[key];
        row_df = (
            name = reactions[i],
            string = reactionstring,
        );
        push!(df, row_df);
    end

    pretty_table(df, tf = tf_simple, alignment = :l)
end

========= =======================================================================================================================
  name     string                                                                                                               
  String   String                                                                                                               
========= =======================================================================================================================
  v1       v1,M_ATP_c+M_L-Citrulline_c+M_L-Aspartate_c,M_AMP_c+M_Diphosphate_c+M_N-(L-Arginino)succinate_c,false
  v2       v2,M_N-(L-Arginino)succinate_c,M_Fumarate_c+M_L-Arginine_c,false
  v3       v3,M_L-Arginine_c+M_H2O_c,M_L-Ornithine_c+M_Urea_c,false
  v4       v4,M_Carbamoyl_phosphate_c+M_L-Ornithine_c,M_Orthophosphate_c+M_L-Citrulline_c,false
  v5       v5,2*M_L-Arginine_c+4*M_Oxygen_c+3*M_NADPH_c+3*M_H_c,2*M_Nitric_oxide_c+2*M_L-Citrulline_c+3*M_NADP_c+4*M_H2O_c,true
 

Update the objective function. 

In [13]:
i = findfirst(x-> x=="b4", model.reactions);
objective = model.objective;
objective[i] = -1; # why negative 1?

__Compute the optimal flux distribution__: Finally, let's compute the optimal metabolic distribution $\left\{\hat{v}_{i} \mid i = 1,2,\dots,\mathcal{R}\right\}$ by solving the [linear programming problem](). We solve the optimization problem by passing the `model::MyPrimalFluxBalanceAnalysisCalculationModel` to [the `solve(...)` method](src/Compute.jl). This method returns a `solution::Dict{String, Any}` dictionary, which holds information about the solution.
* __Why the [try-catch environment](https://docs.julialang.org/en/v1/base/base/#try)__? The [solve(...) method](src/Compute.jl) has an [@assert statement](https://docs.julialang.org/en/v1/base/base/#Base.@assert) to check if the calculation has converged. Thus, the solve method can [throw](https://docs.julialang.org/en/v1/base/base/#Core.throw) an [AssertionError](https://docs.julialang.org/en/v1/base/base/#Core.AssertionError) if the optimization problem fails to converge. To gracefully handle this case, we use a [try-catch construct](https://docs.julialang.org/en/v1/base/base/#try). See the [is_solved_and_feasible method from the JuMP package](https://jump.dev/JuMP.jl/stable/api/JuMP/#JuMP.is_solved_and_feasible) for more information.

In [15]:
solution = let
    
    solution = nothing; # initialize nothing for the solution
    try
        solution = solve(model); # call the solve method with our problem model -
    catch error
        println("error: $(error)"); # Oooooops! Looks like we have a *major malfunction*, problem didn't solve
    end

    # return solution
    solution
end;

__Flux table__: Let's use [the `pretty_tables(...)` method exported by the `PrettyTables.jl` package](https://github.com/ronisbr/PrettyTables.jl) to display the estimated optimal metabolic fluxes. `Unhide` the code block below to see how we constructed the flux table.

In [17]:
let

    # setup -
    S = model.S;
    flux_bounds_array = model.fluxbounds;
    number_of_reactions = size(S,2); # columns
	flux_table = Array{Any,2}(undef,number_of_reactions,5)
    flux = solution["argmax"];
    
    # populate the state table -
	for reaction_index = 1:number_of_reactions
		flux_table[reaction_index,1] = model.reactions[reaction_index]
		flux_table[reaction_index,2] = flux[reaction_index]
		flux_table[reaction_index,3] = flux_bounds_array[reaction_index,1]
		flux_table[reaction_index,4] = flux_bounds_array[reaction_index,2]
        flux_table[reaction_index,5] = model.reactions[reaction_index] |> key-> rd[key]
	end

    # header row -
	flux_table_header_row = (["Reaction","v̂ᵢ", "v̂ᵢ LB", "v̂ᵢ UB", "Reaction"],["","mmol/gDW-time", "mmol/gDW-time", "mmol/gDW-time", "N/A"]);
		
	# write the table -
	pretty_table(flux_table; header=flux_table_header_row, tf=tf_simple, alignment = :l)
end

=========== =============== =============== =============== =======================================================================================================================
  Reaction   v̂ᵢ              v̂ᵢ LB           v̂ᵢ UB           Reaction                                                                                                             
             mmol/gDW-time   mmol/gDW-time   mmol/gDW-time   N/A                                                                                                                  
=========== =============== =============== =============== =======================================================================================================================
  v1         1000.0          0.0             1000.0          v1,M_ATP_c+M_L-Citrulline_c+M_L-Aspartate_c,M_AMP_c+M_Diphosphate_c+M_N-(L-Arginino)succinate_c,false
  v2         1000.0          0.0             1000.0          v2,M_N-(L-Arginino)succinate_c,M_Fumarate_c+M_L-Arginin

## Models for gene expression logic
One of the shortcomings that we discussed about flux balance analysis was:
* __No regulation__. FBA may conflict with experimental data, especially when regulatory loops are excluded. These discrepancies reveal the limitations of relying only on stoichiometric information without considering complex cellular regulation. This can be fixed with [regulatory flux balance analysis](https://pubmed.ncbi.nlm.nih.gov/11708855/). Gene expression is _easy(ish)_, but allosteric regulation (activity) is hard.

Let's investigate how we could describe gene regulation in flux balance analysis. Suppose the flux problem we were interested in was composed of enzymes encoded by the genes $\mathcal{G}=1,2,\dots, N$.
The _action_ of each gene is described by two differential equations, one for mRNA concentration ($m_{j}$, units: `nmol/gDW`) and a second for the corresponding protein concentration ($p_{j}$, units: `nmol/gDW`):
$$
\begin{align*}
	\dot{m}_{j} &= r_{X,j}u_{j}\left(\dots\right) - \left(\theta_{m,j}+\mu\right)\cdot{m_{j}}+\lambda_{j}\quad{j=1,2,\dots,N}\\
	\dot{p}_{j} &= r_{L,j}w_{j}\left(\dots\right) - \left(\theta_{p,j}+\mu\right)\cdot{p_{j}}
\end{align*}
$$
Terms in the balances:
* _Transcription_: The term $r_{X,j}u_{j}\left(\dots\right)$ in the mRNA balance, which denotes the _regulated rate of transcription_ for gene $j$. This is 
the product of a _kinetic limit_ $r_{X,j}$ (units: `nmol/gDW-h`) and a transcription control function $0\leq{u_{j}\left(\dots\right)}\leq{1}$ (dimensionless).
The final term $\lambda_{j}$ is the _unregulated expression rate_ of mRNA $j$ (units: `nmol/gDW-time`), i.e., this is the _leak_ expression rate.
* _Translation_: The _regulated rate of translation_ of mRNA $j$, denoted by $r_{L,j}w_{j}$, is also the product of the
kinetic limit of translation (units: `nmol/gDW-time`) and a translational control term $0\leq{w_{j}\left(\dots\right)}\leq{1}$ (dimensionless).
* _Degradation_: Lastly, $\theta_{\star,j}$ denotes the first-order rate constant (units: `1/time`) governing degradation of protein and mRNA, and $\mu$ is the specific growth rate of the cell (units: `1/time`). We get the latter term using cell-specific concentration units (e.g., `nmol/gDW`).

### Steady-state assumption
We have publically said (without proof _yet_) that gene expression _is slow_ and metabolism _is fast_. This means that the mRNA and protein concentrations are at an approximate steady state, i.e., $\dot{m}_{j}=\dot{p}_{j}=0$ from the perspective of the metabolic network. This allows us to solve the gene expression equations for the steady-state mRNA and protein concentrations. Let's show the steps to compute the steady-state mRNA concentration $m^{\star}_{j}$:
$$
\begin{align*}
r_{X,j}u_{j}\left(\dots\right) - \left(\theta_{m,j}+\mu\right)\cdot{m_{j}}+\lambda_{j} & = \dot{m}_{j}\\
r_{X,j}u_{j}\left(\dots\right) - \left(\theta_{m,j}+\mu\right)\cdot{m^{\star}_{j}}+\lambda_{j} &= 0 \\
r_{X,j}u_{j}\left(\dots\right) + \lambda_{j} & = \left(\theta_{m,j}+\mu\right)\cdot{m^{\star}_{j}}\\
\frac{r_{X,j}u_{j}\left(\dots\right) + \lambda_{j}}{\theta_{m,j}+\mu} &= m^{\star}_{j}\quad\text{for }j=1,2,\dots,N\quad\blacksquare
\end{align*}
$$
Following the same steps, we can compute the steady-state protein concentration $p^{\star}_{j}$:
$$
\begin{equation*}
p^{\star}_{j} = \frac{r_{L,j}w_{j}\left(\dots\right)}{\theta_{p,j}+\mu}\quad\text{for }j=1,2,\dots,N\quad\blacksquare
\end{equation*}
$$

Some things to think about:
* _Nonlinearity_: The expressions for $m^{\star}_{j}$ and $p^{\star}_{j}$ are tricker than they may seem at first blush. The steady-state mRNA and protein concentrations are a function of the kinetic limits of transcription and translation, the control functions, the degradation rates, and the specific growth rate of the cell. At the mRNA level, the $u(...)$ model could be a function of metabolite and protein concentrations. While the kinetic limit of translation $r_{L,j}$ and the $w_{j}(...)$ terms will be functions of the mRNA concentrations and other factors, such as ribosome availability, etc.
* _Models?_ We need to formulate the control functions $u_{j}(...)$ and $w_{j}(...)$, and the degradation rates $\theta_{m,j}$ and $\theta_{p,j}$, and the kinetic limits $r_{X,j}$ and $r_{L,j}$ for each gene in the system. These will be functions of parameters that must be estimated from literature studies or experimental data.
* _Complication_. Finally, the enzymes catalyzing the reactions in the metabolic network are often complexes of different protein subunits, where a different gene encodes each subunit. We need to formulate the gene-protein-reaction (GPR) rules that link the genes to the proteins and the proteins to the reactions in the metabolic network. This will allow us to integrate the gene expression logic into the flux balance analysis problem.

# Today?
That's a wrap! Let's review - what are some things we discussed today?